In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
from os.path import exists
from os import remove
import sklearn.linear_model as lm
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import sklearn.model_selection as ms
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import f_regression

import sys
sys.path.insert(0, "../util/")
import util as util

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# Add Dummies and Train-Test-Split

In [2]:
housingTrainWithDummiesExists = exists('../data/housingTrainWithDummies.csv')
housingTestWithDummiesExists = exists('../data/housingTestWithDummies.csv')
housingTrainWithoutDummiesExists = exists('../data/housingTrainWithoutDummies.csv')
housingTestWithoutDummiesExists = exists('../data/housingTestWithoutDummies.csv')
relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

if (housingTrainWithDummiesExists and housingTestWithDummiesExists and
   housingTrainWithoutDummiesExists and housingTestWithoutDummiesExists and
   relatedDummiesDictionaryExists):
    housingTrainWithDummies = pd.read_csv('../data/housingTrainWithDummies.csv')
    housingTestWithDummies = pd.read_csv('../data/housingTestWithDummies.csv')
    housingTrainWithoutDummies = pd.read_csv('../data/housingTrainWithoutDummies.csv')
    housingTestWithoutDummies = pd.read_csv('../data/housingTestWithoutDummies.csv')
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    
else:
    if (housingTrainWithDummiesExists):
        remove('../data/housingTrainWithDummies.csv')
    if (housingTestWithDummiesExists):
        remove('../data/housingTestWithDummies.csv')
    if (housingTrainWithoutDummiesExists):
        remove('../data/housingTrainWithoutDummies.csv')
    if (housingTestWithoutDummiesExists):
        remove('../data/housingTestWithoutDummies.csv')
    if (relatedDummiesDictionaryExists):
        remove('../data/relatedDummiesDictionary.json')
    
    amesHousing = pd.read_csv('../data/Ames_Housing_Price_Data.csv')   
    
    amesHousing['MSSubClass'] = amesHousing['MSSubClass'].apply(str)
    
    amesHousingCategoricalData = amesHousing.select_dtypes(include=['O'])
    for col in amesHousingCategoricalData.columns:
        amesHousing[col]=amesHousing[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in amesHousingCategoricalData.columns:
        dummyData = pd.get_dummies(amesHousing[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
        amesHousing = pd.concat([amesHousing,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
    
    salesPrice = amesHousing['SalePrice']
    X = amesHousing.drop(['SalePrice'], axis=1)
    
    housingTrainWithDummies, housingTestWithDummies, salesPriceTrain, salesPriceTest = train_test_split(X, salesPrice, test_size=0.2)
    
    housingTrainWithDummies['SalePrice'] = salesPriceTrain
    housingTestWithDummies['SalePrice'] = salesPriceTest
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    housingTrainWithoutDummies = housingTrainWithDummies.drop(dummyColumns, axis=1)
    housingTestWithoutDummies = housingTestWithDummies.drop(dummyColumns, axis=1)
    
    housingTrainWithDummies.to_csv('../data/housingTrainWithDummies.csv')
    housingTestWithDummies.to_csv('../data/housingTestWithDummies.csv')
    housingTrainWithoutDummies.to_csv('../data/housingTrainWithoutDummies.csv')
    housingTestWithoutDummies.to_csv('../data/housingTestWithoutDummies.csv')

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
if (("ISUDistance" not in housingTrainWithoutDummies.columns) or 
    ("ISUDistance" not in housingTrainWithDummies.columns)):
    housingTrainWithoutDummies = util.returnDFWithISUDistance(housingTrainWithoutDummies,True)
    housingTrainWithDummies["ISUDistance"] = housingTrainWithoutDummies["ISUDistance"]
    housingTrainWithDummies.to_csv('../data/housingTrainWithDummies.csv')
    housingTrainWithoutDummies.to_csv('../data/housingTrainWithoutDummies.csv')

# Training

## Linear Regression

In [4]:
housingTrainNumerical = housingTrainWithDummies.select_dtypes(include=['uint8','int64','float64'])

In [5]:
if (exists('../data/trainNanReplacementValuesDictionary.json')):
    with open('../data/trainNanReplacementValuesDictionary.json') as d:
        trainNanReplacementValuesDictionary = json.load(d)
else:
    trainNanReplacementValuesDictionary = {}
    for col in housingTrainNumerical:
        if col in ["YearBuilt","YearRemodAdd","GarageYrBlt","YrSold","ISUDistance"]:
            trainNanReplacementValuesDictionary[str(col)] = housingTrainNumerical[col].mean()
        elif col in ["OverallQual","OverallCond","MoSold"]:
            trainNanReplacementValuesDictionary[str(col)] = round(np.mean(housingTrainNumerical[col].mode().values))
        else:
            trainNanReplacementValuesDictionary[str(col)] = 0
    with open('../data/trainNanReplacementValuesDictionary.json', 'w') as fp:
        json.dump(trainNanReplacementValuesDictionary, fp)
    

In [6]:
housingTrainNumerical = util.replaceNansWithTrainingDataValues(housingTrainNumerical)

../util/util.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(trainNanReplacementValuesDictionary[str(col)])


In [7]:
salesPrice = housingTrainNumerical["SalePrice"]
X = housingTrainNumerical.drop(['SalePrice'], axis=1)

In [8]:
initialAlpha = [1]+list(np.linspace(50,1000,20))

In [9]:
grid_para_elsatic_net_initial = {'alpha': initialAlpha, 'l1_ratio': np.linspace(0,1,11)}

In [10]:
elasticNet = ElasticNet()
if False:
    grid_search_elastic_net = ms.GridSearchCV(elasticNet, grid_para_elsatic_net_initial, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True)
    grid_search_elastic_net.fit(X,salesPrice)
    grid_search_elastic_net.best_params_

In [11]:
finalAlpha = [1] + list(np.linspace(5,50,10))

In [12]:
grid_para_elsatic_net_final = {'alpha': finalAlpha, 'l1_ratio': [0.0]}

In [13]:
grid_search_elastic_net = ms.GridSearchCV(elasticNet, grid_para_elsatic_net_final, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True)
grid_search_elastic_net.fit(X,salesPrice)
grid_search_elastic_net.best_params_

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 780432660910.6396, tolerance: 1180153387.246548
  model = cd_fast.enet_coordinate_descent(


{'alpha': 1, 'l1_ratio': 0.0}

In [14]:
if exists('../data/sigCorrDictionary.json'):
    with open('../data/sigCorrDictionary.json') as d:
        sigCorrDictionary = json.load(d)
else:
    correlationList = []
    pairList = []
    pValList = []
    for i in range(len(X.columns)):
        for j in range(len(X.columns)):
            if i<j:
                iColumn = X.columns[i]
                jColumn = X.columns[j]
                pairList.append(iColumn+'-'+jColumn)
                corrVal,pVal = stats.spearmanr(X[iColumn],
                                              X[jColumn])
                correlationList.append(corrVal)
                pValList.append(pVal)

    amesHousingCorreltaion = pd.DataFrame({"pair":pairList,
                                    "corr":correlationList,
                                    "pVal":pValList
                                   },columns=["pair","corr","pVal"]).sort_values(by=["pVal","corr"],
                                                                                 ascending=[True,False])

    amesHousingSigCorrs = amesHousingCorreltaion[(np.isnan(amesHousingCorreltaion['pVal'])==False) & (amesHousingCorreltaion['pVal'] < 0.05)]
    absRValues = abs(amesHousingSigCorrs['corr'])
    amesHousingSigCorrs = amesHousingSigCorrs[abs(amesHousingSigCorrs['corr']) > absRValues.quantile(.975)]

    sigCorrDictionary = {}
    for col in X.columns:
        correlatedRelations = (amesHousingSigCorrs[amesHousingSigCorrs['pair'].str.contains(col)])["pair"]
        correlatedColumns = [name.replace("-","").replace(col,"") for name in correlatedRelations]
        relatedDummies = []
        for corrCol in correlatedColumns:
            if corrCol in relatedDummiesDictionary.keys():
                relatedDummies = relatedDummies + relatedDummiesDictionary[corrCol]
        correlatedColumns = list(set(correlatedColumns + relatedDummies))
        sigCorrDictionary[col] = correlatedColumns
    with open('../data/sigCorrDictionary.json', 'w') as fp:
        json.dump(sigCorrDictionary, fp)

In [15]:
baseV1 = pd.DataFrame()
prebaseV2 = pd.DataFrame()
baseV2 = pd.DataFrame()

linmodel = lm.LinearRegression()
linmodel.fit(np.log(X[['GrLivArea']]),np.log(salesPrice))
baseV1[f'GrLivArea^{round(linmodel.coef_[0],2)}'] = X['GrLivArea'] ** round(linmodel.coef_[0],2)

linmodel = lm.LinearRegression()
XWithNeighborhood = pd.merge(X,housingTrainWithDummies[["PID","Neighborhood"]],how='left', left_on='PID', right_on='PID')
Nbr_quantile = XWithNeighborhood.groupby('Neighborhood')['GrLivArea'].quantile(0.8)
joint_Nbr_quantile = pd.merge(XWithNeighborhood[['Neighborhood']], Nbr_quantile, how='left', left_on='Neighborhood', right_index=True)
smaller_home = XWithNeighborhood.GrLivArea < joint_Nbr_quantile.GrLivArea

small = []
large = []
for i,isSmall in enumerate(smaller_home):
    if isSmall:
        small.append(XWithNeighborhood.GrLivArea[i])
        large.append(0)
    else:
        small.append(0)
        large.append(XWithNeighborhood.GrLivArea[i])
linmodel.fit(np.log(XWithNeighborhood[["GrLivArea"]])[smaller_home],np.log(salesPrice)[smaller_home.values])
baseV2[f'smallGrLivArea^{round(linmodel.coef_[0],2)}'] = small ** round(linmodel.coef_[0],2)
linmodel.fit(np.log(XWithNeighborhood[["GrLivArea"]])[~smaller_home],np.log(salesPrice)[~smaller_home.values])
baseV2[f'largeGrLivArea^{round(linmodel.coef_[0],2)}'] = large ** round(linmodel.coef_[0],2)

In [16]:
elasticNet = ElasticNet(alpha=1.0,l1_ratio=0.0)
results = cross_validate(elasticNet,baseV1,salesPrice)
np.mean(results['test_score'])

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2190067412565.603, tolerance: 933121559.0304306
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2145135067096.4287, tolerance: 933224897.8421112
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2306966466581.9033, tolerance: 966845617.2990716
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-p

0.5327847844107889

In [17]:
elasticNet = ElasticNet(alpha=1.0,l1_ratio=0.0)
results = cross_validate(elasticNet,baseV2,salesPrice)
np.mean(results['test_score'])

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1973370298686.2163, tolerance: 933121559.0304306
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1909927909886.937, tolerance: 933224897.8421112
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2061435328822.23, tolerance: 966845617.2990716
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-pac

0.5808850518248768

In [18]:
availabilityList = list(X.columns)

In [19]:
availabilityList = util.removeDummiesAndCorrelatedFeaturesFromAvailabilityList(availabilityList,"GrLivArea")

In [20]:
f_value,p_value = f_regression(X,salesPrice)
fValuesSorted = pd.DataFrame({"colName":list(X.columns),
                             "f_value":f_value},
                            columns=["colName","f_value"]).sort_values(by=["f_value"],
                                                                                 ascending=[False])

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


In [21]:
xTrain = baseV2
elasticNet = ElasticNet(alpha=1.0,l1_ratio=0.0)
results = cross_validate(elasticNet,xTrain,salesPrice,return_train_score=True)
trainScore = np.mean(results['train_score'])
testScore = np.mean(results['test_score'])
bestScore = testScore
modelEvolution = pd.DataFrame({"features":["\n".join(list(xTrain.columns))],
                              "train_score":[trainScore],
                              "test_score":[testScore],
                              "test_score_lift":[0.0]},
                             columns = ["features","train_score","test_score","test_score_lift"])
util.pretty_print(modelEvolution)

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1973370298686.2163, tolerance: 933121559.0304306
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1909927909886.937, tolerance: 933224897.8421112
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2061435328822.23, tolerance: 966845617.2990716
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-pac

,features,train_score,test_score,test_score_lift
0,smallGrLivArea^1.0largeGrLivArea^1.24,0.583112,0.580885,0.0


In [25]:
for col in fValuesSorted.colName:
    if col in availabilityList:
        if col in relatedDummiesDictionary.keys():
            newX = pd.concat([xTrain,X[relatedDummiesDictionary[col]]],axis=1)
        else:
            if (util.IsHomoskedastic(X[col],salesPrice)):
                newX = xTrain.copy()
                newX[col] = X[col]
            else:
                newX = util.engineerFeature(xTrain,X[col],str(col))
        if (newX != None):
            elasticNet = ElasticNet(alpha=1.0,l1_ratio=0.0)
            results = cross_validate(elasticNet,newX,salesPrice,return_train_score=True)
            trainScore = np.mean(results['train_score'])
            testScore = np.mean(results['test_score'])
            if testScore > bestScore:
                scoreLift = testScore - bestScore
                bestScore = testScore
                xTrain = newX
                newModel = pd.DataFrame({"features":["\n".join(list(xTrain.columns))],
                              "train_score":[trainScore],
                              "test_score":[testScore],
                              "test_score_lift":[scoreLift]},
                             columns = ["features","train_score","test_score","test_score_lift"])
                modelEvolution = pd.concat([modelEvolution,newModel],axis=0)
                util.pretty_print(modelEvolution)
                availabilityList = util.removeDummiesAndCorrelatedFeaturesFromAvailabilityList(availabilityList,col)

OverallQual
GrLivArea
TotalBsmtSF
1stFlrSF
GarageCars
GarageArea
ExterQual_TA
YearBuilt
FullBath
MasVnrArea
KitchenQual_TA
YearRemodAdd
Foundation_PConc
GarageYrBlt
TotRmsAbvGrd
Fireplaces
BsmtFinSF1
FireplaceQu_nan
BsmtFinType1_GLQ
ExterQual_Gd
BsmtQual_TA
GarageFinish_Unf
Neighborhood_NridgHt
MasVnrType_None
BsmtExposure_Gd
GarageType_Detchd
MSSubClass_60
GarageType_Attchd
WoodDeckSF
Neighborhood_NoRidge
Exterior2nd_VinylSd
Exterior1st_VinylSd
Foundation_CBlock
KitchenQual_Gd
FireplaceQu_Gd
HeatingQC_TA
LotShape_Reg
BsmtFullBath
HalfBath
MSZoning_RM
OpenPorchSF
MasVnrType_BrkFace
BsmtExposure_No
2ndFlrSF
LotArea
PavedDrive_Y
GarageCond_TA
BsmtQual_Gd
RoofStyle_Hip
Neighborhood_StoneBr
CentralAir_Y
SaleType_New
SaleCondition_Partial
MSZoning_RL
GarageQual_TA
MasVnrType_Stone
RoofStyle_Gable
PID
Electrical_SBrkr
MSSubClass_30
GarageType_nan
GarageCond_nan
GarageFinish_nan
GarageQual_nan
HouseStyle_2Story
Neighborhood_OldTown
Neighborhood_NAmes
FireplaceQu_TA
GarageType_BuiltIn
GarageFi

## Random Forest

# Testing

## Linear Regression

## Random Forest